<a href="https://colab.research.google.com/github/javohir-ergashboyev/NLP/blob/main/TF_IDF_vectorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IDF vectorizer (smooth and simple)

In [40]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [2]:
text=[
    'I am Javokhir',
    'I am from Uzbekistan',
    'I am student',
    'I like what you said',
    'Apple is very delicious'
    ]
v=TfidfVectorizer()
output_text=v.fit_transform(text)
v.vocabulary_

{'am': 0,
 'javokhir': 5,
 'from': 3,
 'uzbekistan': 9,
 'student': 8,
 'like': 6,
 'what': 11,
 'you': 12,
 'said': 7,
 'apple': 1,
 'is': 4,
 'very': 10,
 'delicious': 2}

In [3]:
v.get_feature_names_out()
for word in v.get_feature_names_out():
  indx=v.vocabulary_.get(word)
  print(f'{word} {v.idf_[indx]}')

am 1.4054651081081644
apple 2.09861228866811
delicious 2.09861228866811
from 2.09861228866811
is 2.09861228866811
javokhir 2.09861228866811
like 2.09861228866811
said 2.09861228866811
student 2.09861228866811
uzbekistan 2.09861228866811
very 2.09861228866811
what 2.09861228866811
you 2.09861228866811


In [4]:
output_text.toarray()[0]

array([0.55645052, 0.        , 0.        , 0.        , 0.        ,
       0.83088075, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])

In [5]:
import pandas as pd
df=pd.read_csv('ecommerceDataset.csv')
df.columns=['label','text']
print(df.shape)
df.head()

(50424, 2)


,label,text
0,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
1,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
2,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
3,Household,Incredible Gifts India Wooden Happy Birthday U...
4,Household,Pitaara Box Romantic Venice Canvas Painting 6m...


In [6]:
df.label.value_counts()

Household                 19312
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: label, dtype: int64

In [7]:
df_hh=df[df.label=='Household'].sample(8671, random_state=100)
df_bk=df[df.label=='Books'].sample(8671, random_state=100)
df_elec=df[df.label=='Electronics'].sample(8671, random_state=100)
df_cloth=df[df.label=='Clothing & Accessories']
df_new=pd.concat([df_hh,df_bk,df_elec, df_cloth])

In [8]:
df_new.label.value_counts()

Household                 8671
Books                     8671
Electronics               8671
Clothing & Accessories    8671
Name: label, dtype: int64

In [9]:
df_new['label_num']=df_new.label.map({
    'Household':0,
    'Electronics':1,
    'Books':2,
    'Clothing & Accessories':3
    })

In [25]:
df_new.isna().sum()

label        0
text         1
label_num    0
dtype: int64

In [37]:
df_new['text'].dropna(inplace=True)
df_new.isna().sum()

label        0
text         0
label_num    0
dtype: int64

In [36]:
# prompt: drop raw index 39329 from df_new

df_new.drop(39329, axis=0, inplace=True)


In [14]:
df_new.sample(5)

,label,text,label_num
14099,Household,VODOOL High Pressure Soap Foamer Car Washing S...,0
29470,Books,New Heinemann Maths: Addition and Subtraction ...,2
36504,Clothing & Accessories,Fabme Women's Strap (Strap_Tansparent_Free) Re...,3
49458,Electronics,APEXEL 10 In 1 Phone Lens Kit 198 Degree Fishe...,1
48419,Electronics,Sony DVPSR760HP/B DVD Player (Black) Sony SYST...,1


In [38]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df_new.text, df_new.label_num, test_size=0.2,
                                           random_state=100, stratify=df_new.label_num)

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [39]:
clf=Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifcation',KNeighborsClassifier())
])
clf.fit(xtrain,ytrain)
ypred=clf.predict(xtest)
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1735
           1       0.96      0.95      0.95      1734
           2       0.96      0.94      0.95      1734
           3       0.98      0.98      0.98      1734

    accuracy                           0.96      6937
   macro avg       0.96      0.96      0.96      6937
weighted avg       0.96      0.96      0.96      6937

